In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import tqdm

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import *

import xgboost as xgb

seed = 2000
np.random.seed(seed)

import tensorflow as tf
import keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, MaxPooling1D, Dropout, Flatten, Dense, Bidirectional
from tensorflow.keras.preprocessing import sequence

import warnings
warnings.filterwarnings("ignore")

num_i = 4

2023-05-29 18:52:33.856743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [162]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_subtask1_preprocessed_{}.csv'.format(num_i))
print(train_df.head())

               index                                               text  label
0     train_01_0_892  the state alleged they hacked sabata petros ch...      1
1    train_01_1_2714  chale was allegedly chased group about thirty ...      0
2   train_01_10_2619  the farmworkers strike resumed tuesday when th...      1
3  train_01_100_2680  demonstrators have filed for permit hold rally...      1
4  train_01_101_3090  footage the attack which included pregnant wom...      1


In [163]:
dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_subtask1_preprocessed_{}.csv'.format(num_i))
dev_df.head()

dev_df_labels = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask1.csv')
labels = dev_df_labels['label'].values
del(dev_df_labels)
print(labels)
print(dev_df.head())

[1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0
 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0
 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1
 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0
 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0
 0 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0
 0 1 0 1 1 1 0]
               index                                               text
0    train_10_0_2136  the movement was catapulted into the headlines...
1     train_10_1_350  several thousand protesters took the streets a...
2   train_10_10_3104  the protest not just about saving medha life b...
3  train_10_100_1188 

In [164]:
embedding_size = 100
window_size = 2
min_word = 2
down_sampling = 1e-2

fast_text_model_train = FastText(train_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_dev = FastText(dev_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_train.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}".format(num_i))
fast_text_model_train = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}".format(num_i))

fast_text_model_dev.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}".format(num_i))
fast_text_model_dev = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}".format(num_i))

In [165]:
def create_file(create_file, model_file, x):
    fast_text_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([fast_text_model.wv[token] for token in row['text']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(100))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(100)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df

fast_text_train_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_ft_{}.csv'.format(num_i)
fast_text_train_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}'.format(num_i)
fast_text_train_embeddings_df = create_file(fast_text_train_filename, fast_text_train_model_file, train_df)

fast_text_train_embeddings = fast_text_train_embeddings_df.values

print(fast_text_train_embeddings_df.shape)
fast_text_train_embeddings_df.head()


fast_text_dev_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_ft_{}.csv'.format(num_i)
fast_text_dev_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}'.format(num_i)
fast_text_dev_embeddings_df = create_file(fast_text_dev_filename, fast_text_dev_model_file, dev_df)

fast_text_dev_embeddings = fast_text_dev_embeddings_df.values

print(fast_text_dev_embeddings_df.shape)
fast_text_dev_embeddings_df.head()

(3075, 100)
(340, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.065436,0.019627,-0.019218,0.019246,0.005223,-0.058909,0.005642,0.082406,-0.082391,-0.036890,...,-0.005942,0.041117,0.036963,-0.056914,0.097453,-0.018364,-0.024337,0.046799,-0.047586,-0.001964
1,-0.053786,0.018129,-0.012374,0.028447,0.006552,-0.051774,0.013088,0.072879,-0.077181,-0.031455,...,-0.003560,0.046006,0.033148,-0.047711,0.102236,-0.011583,-0.018929,0.051499,-0.044079,-0.004301
2,-0.058167,0.015079,-0.013781,0.028405,0.007915,-0.059294,0.012637,0.078766,-0.081738,-0.031280,...,0.002789,0.044201,0.027809,-0.048893,0.096150,-0.012868,-0.023155,0.046271,-0.053691,-0.010426
3,-0.059094,0.018051,-0.013480,0.021814,0.008642,-0.057071,0.008123,0.074087,-0.079854,-0.033300,...,-0.002901,0.047384,0.032736,-0.049737,0.095013,-0.011365,-0.024717,0.047307,-0.045663,0.000717
4,-0.060764,0.014861,-0.018518,0.020886,0.008679,-0.058670,0.009772,0.086035,-0.085776,-0.034804,...,-0.003310,0.041735,0.034714,-0.053423,0.097148,-0.015153,-0.017781,0.050120,-0.050465,-0.000627


In [153]:
x = fast_text_train_embeddings_df.values
y = train_df['label'].values

x_dev = fast_text_dev_embeddings_df.values
y_dev = labels

mm = MinMaxScaler()
x = mm.fit_transform(x)

# Calculating Classweights
class_weights = compute_class_weight(
    class_weight = "balanced",
    classes = np.unique(y),
    y = y
)
class_weights = dict(zip(np.unique(y), class_weights))

count_0 = np.unique(y, return_counts=True)[1][0]
count_1 = np.unique(y, return_counts=True)[1][1]
estimate = count_0/count_1



cv = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

print(class_weights, estimate)

{0: 1.0596140592694694, 1: 0.9467364532019704} 0.8934729064039408


# XGB

In [196]:
xgb_model = xgb.XGBClassifier(scale_pos_weight=estimate)

parameters = {
            'objective':['binary:logistic'],
            'learning_rate': [0.1, 0.01, 0.001, 0.0001], 
            'max_depth': [6, 7, 8],
            'n_estimators': [1000], #number of trees, change it to 1000 for better results
            'seed': [1337]
        }

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=cv, 
                   verbose=0, refit=True)

clf.fit(x, y)
print(clf.best_params_, clf.best_score_)

y_pred = clf.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))

# MNB

In [9]:
mnb = MultinomialNB()

parameters = {
            'fit_prior': [True, False],
            'class_prior': [None, [0.5, 0.5], [0.6, 0.4], [0.4, 0.6]]
        }

mnb_gsc = GridSearchCV(mnb, parameters, n_jobs=5, 
                   cv=cv, 
                   verbose=0, refit=True)

mnb_gsc.fit(x, y)
print(mnb_gsc.best_params_, mnb_gsc.best_score_)

x_dev = fast_text_dev_embeddings_df.values
y_dev = labels

y_pred = mnb_gsc.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))

{'class_prior': None, 'fit_prior': False} 0.5681300813008131
              precision    recall  f1-score   support

           0       0.46      1.00      0.63       155
           1       0.00      0.00      0.00       185

    accuracy                           0.46       340
   macro avg       0.23      0.50      0.31       340
weighted avg       0.21      0.46      0.29       340

0.0
0.0
0.0


# LSTM

In [166]:
def binary_f1(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall)/(precision + recall + K.epsilon())
    
    return f1_val

x = fast_text_train_embeddings_df.values
y = train_df['label'].values

mm = MinMaxScaler()
x = mm.fit_transform(x)

x = x[:,:,None]

red_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", 
            factor=0.6,
            patience=2, 
            min_lr=0.0001,
            verbose=1)



In [167]:

model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x.shape[1:], return_sequences = True))
model_lstm.add(LSTM(64))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics = [binary_f1])

model_lstm.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_109 (LSTM)             (None, 100, 64)           16896     
                                                                 
 lstm_110 (LSTM)             (None, 64)                33024     
                                                                 
 dense_82 (Dense)            (None, 64)                4160      
                                                                 
 flatten_41 (Flatten)        (None, 64)                0         
                                                                 
 dense_83 (Dense)            (None, 1)                 65        
                                                                 
Total params: 54,145
Trainable params: 54,145
Non-trainable params: 0
_________________________________________________________________


In [168]:
batch_size = 128

model_lstm.fit(x, y,
            batch_size=batch_size,
            epochs=20,
            validation_data=(x_dev, y_dev),
            class_weight=class_weights,
            shuffle=True, 
            callbacks=[red_lr])

Epoch 1/20
25/25 [==============================] - 12s 216ms/step - loss: 0.6941 - binary_f1: 0.4306 - val_loss: 0.6934 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 2/20
25/25 [==============================] - 4s 146ms/step - loss: 0.6934 - binary_f1: 0.1073 - val_loss: 0.6929 - val_binary_f1: 0.7007 - lr: 0.0010
Epoch 3/20
25/25 [==============================] - 4s 144ms/step - loss: 0.6935 - binary_f1: 0.1820 - val_loss: 0.6935 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 4/20
25/25 [==============================] - ETA: 0s - loss: 0.6934 - binary_f1: 0.0000e+00
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0006000000284984708.
25/25 [==============================] - 4s 145ms/step - loss: 0.6934 - binary_f1: 0.0000e+00 - val_loss: 0.6939 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 5/20
25/25 [==============================] - 4s 144ms/step - loss: 0.6935 - binary_f1: 0.0000e+00 - val_loss: 0.6939 - val_binary_f1: 0.0000e+00 - lr: 6.0000e-04
Epoch 6/20
25/25 [====

In [172]:
y_pred = model_lstm.predict(x_dev)
y_pred_final = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_dev, y_pred_final))

11/11 [==============================] - 1s 23ms/step
              precision    recall  f1-score   support

           0       0.46      1.00      0.63       155
           1       0.00      0.00      0.00       185

    accuracy                           0.46       340
   macro avg       0.23      0.50      0.31       340
weighted avg       0.21      0.46      0.29       340



# Bidirectional LSTM

In [169]:
model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(64, input_shape = x.shape[1:], return_sequences=True)))
model_bilstm.add(Bidirectional(LSTM(64)))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'sigmoid'))

model_bilstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics = [binary_f1])
model_bilstm.build(input_shape=x.shape)
model_bilstm.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_45 (Bidirecti  (3075, 100, 128)         33792     
 onal)                                                           
                                                                 
 bidirectional_46 (Bidirecti  (3075, 128)              98816     
 onal)                                                           
                                                                 
 dense_84 (Dense)            (3075, 64)                8256      
                                                                 
 flatten_42 (Flatten)        (3075, 64)                0         
                                                                 
 dense_85 (Dense)            (3075, 1)                 65        
                                                                 
Total params: 140,929
Trainable params: 140,929
Non-t

In [170]:
batch_size = 128

model_bilstm.fit(x, y,
            batch_size=batch_size,
            epochs=20,
            validation_data=(x_dev, y_dev),
            class_weight=class_weights,
            shuffle=True,
            callbacks=[red_lr])

Epoch 1/20
25/25 [==============================] - 30s 367ms/step - loss: 0.6937 - binary_f1: 0.2487 - val_loss: 0.6932 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 2/20
25/25 [==============================] - 7s 261ms/step - loss: 0.6940 - binary_f1: 0.5518 - val_loss: 0.6933 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 3/20
25/25 [==============================] - 7s 261ms/step - loss: 0.6933 - binary_f1: 0.5505 - val_loss: 0.6929 - val_binary_f1: 0.7007 - lr: 0.0010
Epoch 4/20
25/25 [==============================] - 7s 264ms/step - loss: 0.6934 - binary_f1: 0.5543 - val_loss: 0.6930 - val_binary_f1: 0.7007 - lr: 0.0010
Epoch 5/20
25/25 [==============================] - ETA: 0s - loss: 0.6936 - binary_f1: 0.6938
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0006000000284984708.
25/25 [==============================] - 6s 259ms/step - loss: 0.6936 - binary_f1: 0.6938 - val_loss: 0.6929 - val_binary_f1: 0.7007 - lr: 0.0010
Epoch 6/20
25/25 [============================

In [171]:
y_pred = model_bilstm.predict(x_dev)
y_pred_final = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_dev, y_pred_final))

11/11 [==============================] - 3s 36ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       155
           1       0.54      1.00      0.70       185

    accuracy                           0.54       340
   macro avg       0.27      0.50      0.35       340
weighted avg       0.30      0.54      0.38       340

